In [ ]:
# URLs
# PK  HOST
# PK  LINK  Visited  FK REGDATE
#     네이버 검색 결과 들어갈 것
#     (네이버 블로그 어디서부터 왔는지 모름
#     그래서 FK 넣어줌)

#     네이버 블로그1 FK = 1
#     네이버 블로그1 FK = 1
# 이때, count(FK)시 1번 페이지가 갖는 전체 link의 수가 될 것
# PR = inbound/전체 link 
# Group by LINK WHERE FK = 1 ==> 1번 사이트에서 온 블로그들 return



In [ ]:
# 전체적인 scheme은
# LINK 
# PK        LINK        Visited         FK          inbound         REGDATE
# 1     /search?q=수지                   1              0
# 2     /id/303030                       2              1
# 3     /id/123123                       2              1

# 여기서 FK는 어떤 host에서 왔는지 의미함
# inbound는 어디서 왔는지 의미함 / 가중치를 의미하기도 함
# REGDATE - 언제 긁었는지
# Y/N - 긁었는지 안긁었는지
# ex)
# LINKS WHERE Y/N = 'N' ORDER BY PK LIMIT 0,N

# URLs
# PK        HOST
# 1         네이버 검색 결과
# 2         네이버 블로그


In [ ]:
# 네이버
# 먼저 url에서 공통적으로 겹치는 애들이 있음
# 쪼개자
# 그랬을때 2번쨰 cell에서 아래의 scheme이 나오게 되며 link도 바뀌게 된다
#

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [ ]:
cur.executescript('''
    CREATE TABLE URLs(
        PK INTEGER PRIMARY KEY,
        SCHEME TEXT NOT NULL DEFAULT 'http',
        NETLOC TEXT NOT NULL,
        REGDATTE TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP      
    );
''')

In [ ]:
cur.execute('INSERT INTO URLs(NETLOC) VALUES("www.naver.com")')

In [ ]:
cur. execute('SELECT * FROM URLs')
cur.fetchall()

In [ ]:
cur.execute('DELETE FROM URLs')

In [ ]:
cur.executescript('''
    DROP TABLE IF EXISTS URLs;
    CREATE TABLE URLs(
        PK INTEGER PRIMARY KEY,
        SCHEME TEXT NOT NULL DEFAULT 'http',
        NETLOC TEXT NOT NULL,
        REGDATTE TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP      
    );
                  
    DROP TABLE IF EXISTS LINKs;
    CREATE TABLE LINKs(
        PK INTEGER PRIMARY KEY,
        PATH TEXT NOT NULL DEFAULT '/',
        QS TEXT ,
        FK INTEGER NOT NULL, => 도메인떔에 어느 도메인에 속했는지
        REF INTEGER NOT NULL, => 이전 도메인 outbound, inbound 링크의 출처가 된다
        VISITED INTEGER NOT NULL DEFAULT 0, => page 방문 여부
        REGDATTE TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP      
    );
''')
# 어디로부터 왔는지 알아야함

In [ ]:
from requests.compat import urlparse

# urls table 검사해서 PK 찾는 구문/ 필요시에 insert도 함
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%88%98%EC%A7%80'
url_seg = urlparse(url)

cur.execute('SELECT PK FROM URLs WHERE NETLOC=?', (url_seg.netloc,))
temp = cur.fetchone()

if not temp:
    cur.execute('INSERT INTO URLs (SCHEME, NETLOC) VALUES(?,?)', url_seg[:2])

cur.execute('SELECT PK FROM URLs WHERE NETLOC=?', (url_seg.netloc,))
PK = cur.fetchone()[0]

In [ ]:
cur.execute('SELECT * FROM URLs')
cur.fetchall()

In [ ]:
# PK를 찾았으니 link에 넣고 link에서도 검사함
cur.execute('SELECT COUNT(PK) FROM LINKs WHERE FK=? AND PATH=?', (PK, url_seg.path))
if cur.fetchone()[0] == 0:
    cur.execute('INSERT INTO LINKs(PATH, QS, FK) VALUES (?, ?, ?)', (url_seg.path, url_seg.query, PK))
    # 본래는 commit 까지 해야하지만 지금 memory에서 작업하므로 없어도 됨

In [ ]:
cur.execute('SELECT * FROM LINKs')
cur.fetchall()
# 한개만 들어가는게 맞음
# 여러개 들어가면 조건을 잘못 만들어놓은 것

In [ ]:
from requests.compat import urljoin, urlunparse
from requests import get
from bs4 import BeautifulSoup
import re

# 여기 파일 보고 고쳐야함

cur.execute('''
            SELECT SCHEME, NETLOC, PATH, QS FROM URLs
            INNER JOIN LINKs
            ON LINKs.FK = URLs.PK
            WHERE VISITED=0
            ORDER BY LINKs.REGDATTE ASC
            LIMIT 0, 10
            ''')

urls = cur.fetchall()
while True:
        # asc => queue의 형태로 한번에 최대 10개 페이지 수집
        cur.execute('''
                SELECT B.PK, B.FK, A.SCHEME, A.NETLOC, B.REF, B.PATH, B.QS FROM URLs AS A
                INNER JOIN LINKs AS B
                ON B.FK = A.PK
                WHERE B.VISITED=0
                ORDER BY B.REGDATTE ASC
                LIMIT 0, 10
                ''')
        urls = cur.fetchall()
        # A.PK => 어느 사이트로부터 왔는가
        # B.PK => 수집했는지 여부 (UPDATE 시 필요)
        # B.FK => 
        if len(urls) == 0:
                break

        while urls:
                url_seg = urls.pop(0)
                url = urlunparse((url_seg[3], url_seg[4], url_seg[5], None, url_seg[6], None))
                
                resp = get(url)
                cur.execte('UPDATE LINKs SET VISITED = 1 WEHRE PK = ?', (url_seg[1],))
                con.commit()

                if resp.status_code == 200 and re.search(r'text/html', resp.headers['content-type']):
                        dom = BeautifulSoup(resp.text)
                        for tag in dom.select('a[href], iframe[src]'):
                                link = tag.attrs['href' if tag.hass_attr('href') else 'src']
                                newLink = urljoin(url, link)

                                newlink_seg = urlparse(newLink)
                                cur.execute('SELECT PK FROM URLs WHERE NETLOC=?', (newlink_seg.netloc,))
                                temp = cur.fetchone()

                                if not temp:
                                        cur.execute('INSERT, INTO URLs (SCHEME, NETLOC) VALUES(?,?)', newlink_seg[:2])
                                cur.execute('SELECT PK FROM URLs WHERE NETLOC=?', (url_seg.netloc,))
                                temp = cur.fetchone()

                                if not temp:
                                        cur.execute('INSERT INTO URLs (SCHEME, NETLOC) VALUES(?,?)', url_seg[:2])

                                cur.execute('SELECT PK FROM URLs WHERE NETLOC=?', (url_seg.netloc,))
                                PK = cur.fetchone()[0]
                        
# 지금 이름이 달라서 그냥 불러오는데 원래는 DOT NOTATION으로 가져오ㅓㅏ야함

In [ ]:
cur.execute('SELECT * FROM LINKs WHERE VISITED=0')
cur.fetchall() # => urls

cur.execute('SELECT * FROM LINKs WHERE VISITED=1')
cur.fetchall() # => visited

In [ ]:
cur.execute('''
            SELECT COUNT(*) FROM LINKs WHERE FK = 1
            ''')
cur.fetchall()[0]

In [ ]:
cur.execute('''
            SELECT COUNT(*) FROM LINKs WHERE REF = 1
            ''')
cur.fetchall()[0]
# 1번 페이지로 URL 개수
# 1번 도메인의 outbound link의 수

In [ ]:
# 이떄, 각 가중치에 관하여 확률을 구하느데
# 각 수식이 존재함
cur.close()

In [ ]:
# ORM
# 이제는 class로 만들어서 넣을 수 있음
# 더이상 SQL문은 필요가 없음
# class를 선언 시에는 create_engine의 dialogue를 통해서 해석된다
# base를 상속받아서 class 선언시에 사용해야한다
# session을 통해서 관리함
# 객체로 표현된 애와 실제 db 사이에 왔다갔다해야하는 모든 경우에
# 주시하다가 우리를 대신 왔다갔다함
# session에 등록된 객체는 table에 왔다갔다가 하는 것을 session이 관리함
# pending이 존재 이것을 풀어줘야하는데 이것을 commit()을 통해서 푼다

In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.schema import Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.orm.session import sessionmaker

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [ ]:
base = declarative_base()

In [ ]:
class User(base):
    __tablename__ = 'T_USER'
    __table_args__ = {'extend_existing':True}   # 이거 기술 문서 보면서 원하는 것 집어넣으면 됨

    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text, nullable=False)

    def __repr__(self):
        return f'pk={self.pk}, name = {self.name}'
    # class의 instance호출 == DB랑 아무 상관 없
    # 얘는 base에 등록된 user라고 하는 class 이다

In [ ]:
base.metadata.tables
# orm의 table 객체를 의미함

In [ ]:
base .metadata.create_all(engine)
# table 객체를 sql문으로 해석해서 table 생성하는 역할
# 즉, table 생성 시 객체를 db에 반영함

In [ ]:
# Mapping:base            ORM:core            DB
# declarative_base        Meta                RDBMS
# class(base) ->          table 객체  ->      table 생김
# 오늘은 mapping:base를 본다

In [ ]:
a = User(name='아무개')
# 아직 db에 안들어갔으므로 pk값이 없음
# 이때, session maker를 통해 sessioln 만들고 bound 할 것임

In [ ]:
session = sessionmaker(engine)
sess = session()

In [ ]:
sess.add(a)
# 여기서 add는 session 객체가 주시하고 있는 class의 instance들을 등록하는 과정이다
# 그래서 sess.dirty, sess.ismodified(a) -> 무언가 변화했을때 dirty에 기록하고 있음 / 비어있으면 변화 없음
# ismodified -> 특정 객체가 어떻게 되었는지 의미함 

In [ ]:
sess.commit()
# 이때, insert를 함
# 이 후에는 ismodified가 False로 바뀌므로 sync가 맞다는 의미이다
# 이 후 a를 호출하면 db에 저장되어서 pk가 나올 수 있다

In [ ]:
a.pk

In [ ]:
a.name = "이름 바꿈"

In [ ]:
sess.dirty

In [ ]:
sess.commit()
# 알아서 update 구문이 돌아감

In [ ]:
sess.query()
# query를 불러옴 대응되는 객체로 가져옴
# 여기서 userㄹ르 가져와보자

In [ ]:
sess.query(User).all()
# 전체 데이터를 가져올 것
sess.query(User).all()[0] is a

In [ ]:
# filter를 입혀보자
sess.query(User).filter(User.pk == 1).all()[0] is a
# sess.query(User).all()[0] => User라는 객체다, 따라서 dot notation으로 값을 불러올 수 있다

In [ ]:
base.registry.dispose()
# base에 등록된 class를 날리는 것

In [ ]:
base.metadata.tables

In [ ]:
from sqlalchemy.schema import Table

class User(base):
    __table__ = Table('T_USER', base.metadata, reflect=True)
    # base.metadata.tables['T_USER']
    # 위의 코드 실행 후 돌아오면 에러가 날것임
    # 이 전에 무조건 reflect로 다시 가져와야함
    __table_args__ = {'extend_existing': True} 
    # reflect (autoload)를 하는 순간 pk랑 name을 그대로 들고옴
    # 만약에 pk를 가져오고싶으면 autoload를 지우고

    # pk = Column('PK', base.metadata.tables['T_USER'].c.PK)

In [ ]:
class Address(base):
    __tablename__ = 'T_ADDRESS'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key = True)
    address = Column('ADDRESS', Text)
    fk = ForeignKey(User.PK)

In [ ]:
base.metadata.create_all(engine)

In [ ]:
a = sess.query(User).one()
ad = Address(address='주소1', fk=a.PK)
sess.add(ad)

In [ ]:
sess.dirty, sess.is_modified(ad)

In [ ]:
sess.commit()

In [ ]:
sess.close()

In [ ]:
engine.dispose()

In [ ]:
base = declarative_base()
engine = create_engine('sqlite:///:memory:')

class User(base):
    __tablename__ = 'T_USER'
    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text, nullable=False)

class Address(base):
    __tablename__ = 'T_ADDRESS'
    pk = Column('PK', Integer, primary_key = True)
    address = Column('NAME', Text, nullable=False)
    fk = Column('FK', Integer, nullable=False)

In [ ]:
base.metadata.create_all(engine)

In [ ]:
session= sessionmaker(engine)
sess = session()

In [ ]:
sess.add(User(name="이름1"))

In [ ]:
a = sess.query(User).one()

In [ ]:
sess.add(Address(address='주소1', fk=a.pk))

In [ ]:
sess.query(Address).all()[0].fk

In [ ]:
from sqlalchemy.sql import join

# 바뀐건 instance로 들어간다는 것 (관계랑 query params에)

sess.query(User.name, Address.address).select_from(join(User, Address, User.pk == Address.fk)).all()

In [ ]:
class Phone(base):
    __tablename__ = 'T_PHONE'
    pk = Column('PK', Integer, primary_key = True)
    phone = Column('PHONE', Text)
    fk = ForeignKey(User.pk)

In [ ]:
base.metadata.create_all(engine)

In [ ]:
sess.add(Phone(phone='전화번호', fk = a.pk))
sess.commit()

In [ ]:
sess.close_all()
base.metadata.clear()
engine.dispose()

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker(engine)
sess = session()

In [ ]:
base = declarative_base()

class User(base):
    __tablename__ = 'T_USER'
    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text, nullable=False)

class Address(base):
    __tablename__ = 'T_ADDRESS'
    pk = Column('PK', Integer, primary_key = True)
    address = Column('NAME', Text, nullable=False)
    fk = Column('FK', Integer, nullable=False)

class Phone(base):
    __tablename__ = 'T_PHONE'
    pk = Column('PK', Integer, primary_key = True)
    phone = Column('PHONE', Text)
    fk = Column('FK', Integer, ForeignKey(User.name))

In [ ]:
base.metadata.create_all(engine)

In [ ]:
a = User(name = "이름1")
sess.add(a)
sess.commit()

In [ ]:
sess.add(Address(address="주소1", fk = a.pk))
sess.commit()

In [ ]:
sess.query(User.name, Address.address).join(Address).all()

In [ ]:
#                                   declrative_base가 다 들고 있음
#                   base.metadata.table     base.registry
#                   base.metadata.clear()   base.registry.dispose()
#         얘는 db에서 reflect해서 받아와야함        => class를 날림 table 객체는 살아있음 / 따라서, table 객체에서 불러오는 것이 가능
#                   객체 자체를 날리기 때문임    
#  database         ORM-CORE                ORM:mapper
# T_USER            Table('T_USER')         User(base)
# T_ADDRESS         Table('T_ADDRESS')      Address(base)

# join()을 보면 원래는 join(T,T,on = reference/ForeignKey X)
# 이미 table들의 관계가 있음 그래서 
# ON절 없애고 사용가능함 => fk 옵션을 주었으므로 (객체의 props를 주었음) on절이 생략 가능하다 (자동으로 관계를 의미하므로)
# 예를 들어
# Left join (Right)
# ORM.pdf 보면 됨

In [ ]:
sess.close()
base.metadata.clear()
engine.dispose()

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker(engine)
sess = session()

In [ ]:
from sqlalchemy.orm import relationship

class Artist(base):
    __table__ = 'T_ARTIST'
    __table_args__ = {'extent_existings': True}
    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text)
    albums = relationship('Album', back_populates='artists', uselist=True)

    def __repr__(self):
        return f'pk={self.pk}, name = {self.name}'

class Album(base):
    __table__ = 'T_ALBUM'
    __table_args__ = {'extent_existings': True}
    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text)
    artists = relationship('Artist', back_populates='albums')
    # back_populates를 사용하여 찾아갈 것이다

    def __repr__(self):
        return f'pk={self.pk}, name = {self.name}'

class Genre(base):
    pass
class Track(base):
    pass

In [ ]:
class Artist(base):
    __tablename__ = 'T_ARTIST'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text)
    albums = relationship('Album', back_populates='artists')

    def __repr__(self):
        return f'pk={self.pk}, name = {self.name}'

In [ ]:
class Album(base):
    __tablename__ = 'T_ALBUM'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key = True)
    fk = Column('FK', None, ForeignKey(Artist.pk))
    name = Column('NAME', Text)
    artists = relationship('Artist', back_populates='albums', uselist=True)
    # back_populates를 사용하여 찾아갈 것이다

    def __repr__(self):
        return f'pk={self.pk}, name = {self.name}'

In [ ]:
base.registry.dispose()
base.metadata.clear()

In [ ]:
base.metadata.create_all(engine)
a = Artist(name = "가수1")
sess.add(a)
sess.commit()

In [ ]:
a = sess.query(Artist).one()
a.albums.append(Album(name='가수1의 앨범1'))
# 이런식으로 append 가능하고 commit을 잊지말것

In [ ]:
sess.close()
base.registry.dispose()
base.metadata.clear()
engine.dispose()

# engine 새롭게 만들고, session 맞춰서 열고
# base 상속받아서 class 만들고
# existing 수정 시에 class 덮어씌우는 option
# relation은 어느 class랑 관계 맺고 있는지
# back_populates는 반대로 돌아올 때 어떤 것을 가지고 와야하는지
# 이후 a.albums.append()까지하면 instance에만 들어가고 db에 적용되지 않았음
# 따라서, commit 해줘야함